# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [40]:
%reload_ext dotenv
%dotenv

In [41]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [45]:
import os
from glob import glob

price_data_dir = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join(price_data_dir, "**/*.parquet"), recursive=True)

In [46]:
print(f"PRICE_DATA directory: {price_data_dir}")
print(f"Found {len(parquet_files)} parquet files.")

PRICE_DATA directory: ../../05_src/data/prices/
Found 60 parquet files.


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# load all parquet files into a single Dask DataFrame
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

def create_features(df):

    df = df.sort_values(by='Date')

    # add lags for Close and Adj Close
    df = df.assign(
        Close_lag_1 = df['Close'].shift(1),
        Adj_Close_lag_1 = df['Adj Close'].shift(1)
    )

    # add returns: (Close / Close_lag_1) - 1
    df['returns'] = (df['Close'] / df['Close_lag_1']) - 1

    # add hi_lo_range: High minus Low
    df['hi_lo_range'] = df['High'] - df['Low']

    return df

meta_df = dd_px._meta.copy()
meta_df['Close_lag_1'] = float
meta_df['Adj_Close_lag_1'] = float
meta_df['returns'] = float
meta_df['hi_lo_range'] = float


# apply the feature creation function to each ticker group
dd_feat = dd_px.groupby('ticker', group_keys=False).apply(
    create_features,
    meta=meta_df
)

In [55]:
print(dd_feat.head())

             Date   Open   High    Low  Close  Adj Close      Volume   source  \
ticker                                                                          
ACN    2001-07-19  15.10  15.29  15.00  15.17  11.404394  34994300.0  ACN.csv   
ACN    2001-07-20  15.05  15.05  14.80  15.01  11.284108   9238500.0  ACN.csv   
ACN    2001-07-23  15.00  15.01  14.55  15.00  11.276587   7501000.0  ACN.csv   
ACN    2001-07-24  14.95  14.97  14.70  14.86  11.171341   3537300.0  ACN.csv   
ACN    2001-07-25  14.70  14.95  14.65  14.95  11.238999   4208100.0  ACN.csv   

        Close_lag_1  Adj_Close_lag_1   returns  hi_lo_range  
ticker                                                       
ACN             NaN              NaN       NaN         0.29  
ACN           15.17        11.404394 -0.010547         0.25  
ACN           15.01        11.284108 -0.000666         0.46  
ACN           15.00        11.276587 -0.009333         0.27  
ACN           14.86        11.171341  0.006057         0.30 

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [58]:
import pandas as pd

df_feat = dd_feat.compute()

df_feat = df_feat.reset_index()

df_feat['ma_returns_10d'] = (
    df_feat
    .groupby('ticker', observed=True)['returns']
    .rolling(10, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

In [59]:
# Optional: Display the end of the DataFrame to verify the new feature
print(df_feat[['ticker', 'Date', 'returns', 'ma_returns_10d']].tail(20))

       ticker       Date   returns  ma_returns_10d
239639   ZIXI 2020-03-05 -0.029021        0.001838
239640   ZIXI 2020-03-06 -0.067248       -0.016526
239641   ZIXI 2020-03-09 -0.101469       -0.028130
239642   ZIXI 2020-03-10  0.026746       -0.022251
239643   ZIXI 2020-03-11 -0.133140       -0.035223
239644   ZIXI 2020-03-12 -0.185309       -0.045392
239645   ZIXI 2020-03-13 -0.030738       -0.046965
239646   ZIXI 2020-03-16 -0.103594       -0.060751
239647   ZIXI 2020-03-17 -0.075472       -0.065108
239648   ZIXI 2020-03-18 -0.227041       -0.092628
239649   ZIXI 2020-03-19  0.095710       -0.080155
239650   ZIXI 2020-03-20  0.069277       -0.066503
239651   ZIXI 2020-03-23 -0.090141       -0.065370
239652   ZIXI 2020-03-24  0.142415       -0.053803
239653   ZIXI 2020-03-25  0.084011       -0.032088
239654   ZIXI 2020-03-26  0.127500       -0.000807
239655   ZIXI 2020-03-27  0.019956        0.004262
239656   ZIXI 2020-03-30  0.008696        0.015491
239657   ZIXI 2020-03-31 -0.071

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, as Dask DataFrames support the groupby().rolling().mean() operation, making the calculation technically possible within Dask. It would be better running it in Dask if the data set is too large to fir into the memory of a single machine.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.